# IMPORT PARAMETERS AND LIBRARIES

In [120]:
apk_path='Repack_APKs'
decompile_path='./Decompiled_files'
dex2jar='./dex2jar-2.0/d2j-dex2jar.sh'
jd_cli='./jd-cli-1.1.0.Final-dist/jd-cli.jar'

In [121]:
import pandas as pd
import wget
from os import listdir
from os.path import isfile, join
import timeit

In [122]:
import subprocess

# GET FILE LISTS

In [123]:
df=pd.read_csv('./repackaging_pairs.txt',sep=',')
df_sub=df.iloc[3000:6000]
file_lst=list(df_sub.iloc[:,0])+list(df_sub.iloc[:,1])
file_lst=list(set(file_lst))
file_lst.sort()
mypath='Repack_APKs/'
downlaod_lst=[f for f in listdir(mypath) if isfile(join(mypath, f))]
downlaod_lst=[a for a in downlaod_lst if a.split('.')[1]=='apk']
downlaod_lst=[a[0:a.index('.apk')] for a in downlaod_lst]

print('file count not downloading =',len(set(file_lst)-set(downlaod_lst)))

file count not downloading = 0


In [124]:
xml_lst=listdir(os.path.join(decompile_path,'xml_files'))
xml_lst=[a[0:a.index('.xml')] for a in xml_lst if '.xml' in a]

print('file count not decompiling =',len(set(file_lst)-set(xml_lst)))

file count not decompiling = 3713


In [125]:
file_lst=list(set(file_lst)-set(xml_lst))
file_lst.sort()

# DECOMPILE

In [130]:
def command(line):
#     f=os.popen(line)
#     f.readlines()
    proc=subprocess.Popen(line, shell=True) 
    try:
        outs, errs = proc.communicate(timeout=20)
    except subprocess.TimeoutExpired:
        proc.kill()
        print(line)
        outs, errs = proc.communicate()

In [ ]:
## f=os.popen(f'sudo chmod +x {dex2jar}/d2j_invoke.sh')
f.readlines()
start = timeit.default_timer()
for i,file in enumerate(file_lst):
    if i%10==0 and i!=0:
        stop = timeit.default_timer()
        print('Time: %s s'% str(stop - start))  
        start = timeit.default_timer()
    
    apk_file=file+'.apk'
    apk_file_path=os.path.join( apk_path,  apk_file)
    
    output_jar=os.path.join(decompile_path, apk_file+'.jar')
    command(f'sh {dex2jar} -f -o {output_jar} {apk_file_path}')
#     break
    
    java_code_folder1 = os.path.join(decompile_path, file)
    os.mkdir(java_code_folder1)
    command(f'java -jar {jd_cli} -od {java_code_folder1} {output_jar}')

    java_code_folder2 = os.path.join(decompile_path, file+'_p2')
    command(f'apktool d {apk_file_path} -o {java_code_folder2}')
    
    command(f'cp {java_code_folder2}/AndroidManifest.xml {decompile_path}/xml_files/{file}.xml')
    
    command(f'rm -rf {output_jar}')
    command(f'rm -rf {java_code_folder2}')
#     command(f'rm -rf {java_code_folder1}')
